In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

In [ ]:
#Create the dataframe
weather_data_df = pd.read_csv("weather_data.csv")
weather_data_df.head()

In [ ]:
locations = weather_data_df[['lat','lon']]
weight = weather_data_df['humidity']

m = gmaps.figure()
m.add_layer(gmaps.heatmap_layer(locations, weights=weight,dissipating=False,max_intensity=.5,point_radius=10))
m

In [ ]:
#We are going to be finding a location spot so we need to make some filters
max_temp = weather_data_df['temp'] <= 75
min_temp = weather_data_df['temp'] >= 65
windspeed = weather_data_df['wind_speed'] <=10
cloudiness = weather_data_df['cloud_cover'] <=20
hot_wet = weather_data_df['humidity'] <= 50

#Now to apply the filters
vacay_df = weather_data_df[max_temp & min_temp & windspeed & cloudiness & hot_wet]
vacay_df

In [ ]:
#Next we are going to find the closest hotels to the city center

# set up a parameters dictionary
params = {
    "radius": 5000,
    "keyword": "hotels",
    "type": "hotel",
    "key": g_key
}

for index, row in vacay_df.iterrows():
    
    #add the values for lat and long
    lat = row["lat"]
    lng = row["lon"]
    params["location"] = f"{lat},{lng}"

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params) 
    hotel_name = response.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    try:
        vacay_df.loc[index, "Hotel Name"] = hotel_name["results"][0]['name']
    except (KeyError,IndexError):
        print("No hotels within your selected radius, Skipping")
vacay_df

In [ ]:
#Now I am going to make a new df and drop the NA's
hotel_df = vacay_df.copy()
hotel_df = hotel_df[hotel_df['Hotel Name'].notna()]
hotel_df

In [ ]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lon"]]

In [ ]:
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
m.add_layer(markers)
m